In [32]:
import re
from jamo import h2j, j2h, j2hcj
from konlpy.tag import Kkma, Komoran

kkma = Kkma()
komoran = Komoran()
tokenizer = kkma

In [33]:
# '-(으)ㅁ'

# 종성이 있는 경우 음
# 종성이 없는 경우 ㅁ

# 활용으로 인해 종성이 탈락하더라도 종성이 있는 경우로 취급

# ㄷ 불규칙 활용
# 어간 받침 'ㄷ'이 홀소리로 시작되는 어미 앞에서 'ㄹ'로 바뀌는 활용 형식이다.
# 걷다[步] → 걸어, 걸으니
# 걷 > 걸 > 걸음 <<< 종성 있는 경우

# ㅅ 불규칙 활용
# 어간 끝소리 'ㅅ'이 홀소리로 시작하는 어미 앞에서 사라지는 활용 형식이다.
# 긋다 → 그어, 그으니
# 긋 > 그 > 그음 <<< 종성 있는 경우

# ㅂ 불규칙 활용
# 어간 끝소리 'ㅂ'이 '우'로 바뀌는 활용 형식이다. '워', '우니'의 형태로 결합되어 나타난다.
# 가깝다 → 가까워, 가까우니
# 가깝 > 가까우 > 가까움 <<< 종성 없는 경우

In [34]:
irregular_conjugation_stems = ['걷', '긷', '깨닫', '눋', '닫', '듣', '묻', '붇', '싣', '일컫', '가깝', '가볍', '간지럽', '굽', '그립', '깁', '껄끄럽', '노엽', '더럽', '덥', '맵', '메스껍', '무겁', '반갑', '부끄럽', '사납', '서럽', '쑥스럽', '줍', '긋', '낫', '붓', '잇', '잣', '젓', '짓', ]

In [35]:
def verb_exception_handler(verb):
    if verb == '나서':
        return '나'
    else:
        return verb

In [36]:
def verb_to_noun(verb):
    
    """
    한국어의 용언을 명사형으로 변환하는 함수입니다.
    불규칙 활용이 반영되어 있으나 예외의 경우 아직 사전 구축이 되어 있지 않습니다.
    
    """
    
    verb = verb_exception_handler(verb)

    eum = '음'  # 명사형 전성 어미 '음'
    m = h2j('음')[-1]  # 명사형 전성 어미 'ㅁ'

    d = h2j('ㄷ')  # ㄷ 불규칙 활용
    s = h2j('ㅅ')  # ㅅ 불규칙 활용
    b = h2j('ㅂ')  # ㅂ 불규칙 활용

    last_syllables = h2j(verb[-1])  # 용언의 마지막 자모군
    final = j2hcj(last_syllables[-1])  # 마지막 자모군의 끝글자

    pattern = re.compile(r'[ㅏ-ㅣ]')
    has_final = not bool(pattern.match(final))  # 마지막 자모군에 종성 존재 여부

    if has_final and final != b:  # 종성이 존재하는 경우 / ㅂ 불규칙 활용의 경우 종성이 없는 경우로 취급
        if final == d and verb in irregular_conjugation_stems:  # ㄷ 불규칙 활용 적용
            modified = last_syllables[:-1] + h2j('ㄹ')
            return verb[:-1] + j2h(*modified) + eum
        elif final == s and verb in irregular_conjugation_stems:  # ㅅ 불규칙 활용 적용
            modified = last_syllables[:-1]
            return verb[:-1] + j2h(*modified) + eum
        else:  # 종성이 존재하나 불규칙 활용이 아닌 경우
            modified = last_syllables
            return verb[:-1] + j2h(*modified) + eum
    else:  # 종성이 존재하지 않는 경우
        if final == b and verb in irregular_conjugation_stems:  # ㅂ 불규칙 활용
            modified = h2j('우') + m
            return verb[:-1] + j2h(*last_syllables[:-1]) + j2h(*modified)
        else:  # 종성이 존재하지 않으며 불규칙 활용이 아닌 경우
            modified = last_syllables + m
            return verb[:-1] + j2h(*modified)

# 보다 / 돌다 / 걷다 / 긋다 / 가깝다 / 반갑다 / 넘어지

print(verb_to_noun('보'))
print(verb_to_noun('돌'))
print(verb_to_noun('걷'))
print(verb_to_noun('긋'))
print(verb_to_noun('가깝'))
print(verb_to_noun('반갑'))
print(verb_to_noun('넘어지'))

봄
돌음
걸음
그음
가까움
반가움
넘어짐


In [37]:
def extract_keywords(sentence):
    
    """
    문장을 입력으로 받아서 형태소 분석을 수행하고, 명사(Noun)와 동사(Verb)를 추출하는 함수

    """
    pos = tokenizer.pos(sentence)
    keywords_w_pos = [[word, tag] for word, tag in pos if tag in ['NNG', 'NNP', 'VV', 'VA']]
    return keywords_w_pos

# 
def sentence_to_noun_verb(sentence):

    """
    "사다리가 쓰러져서" 문장을 "사다리 쓰러짐" 문자열로 변환하는 함수

    """
    keywords_w_pos = extract_keywords(sentence)
    for item in keywords_w_pos:
        if item[1] == 'VV' or item[1] == 'VA':
            item[0] = verb_to_noun(item[0])
    result = [word for word, tag in keywords_w_pos]
    return ' '.join(result)

In [38]:
test_cases = ['발판이 쓰러져서', '사다리가 넘어지다', '사다리가 넘어져서', '사다리가 넘어져', '발판이 쓰러져', '화재 발생', '배탈이 나서', '배탈이 났다']

for item in test_cases:
    result = extract_keywords(item)
    print(result)

for item in test_cases:
    result = sentence_to_noun_verb(item)
    print(result)

[['발판', 'NNG'], ['쓰러지', 'VV']]
[['사다리', 'NNG'], ['넘어지', 'VV']]
[['사다리', 'NNG'], ['넘어지', 'VV']]
[['사다리', 'NNG'], ['넘어지', 'VV']]
[['발판', 'NNG'], ['쓰러지', 'VV']]
[['화재', 'NNG'], ['발생', 'NNG']]
[['배탈', 'NNG'], ['나서', 'VV']]
[['배탈', 'NNG'], ['나', 'VV']]
발판 쓰러짐
사다리 넘어짐
사다리 넘어짐
사다리 넘어짐
발판 쓰러짐
화재 발생
배탈 남
배탈 남
